# Graphistry OWASP Amass Tutorial

Map the footprint of your network

* **Amass**: "The OWASP Amass tool suite obtains subdomain names by scraping data sources, recursive brute forcing, crawling web archives, permuting/altering names and reverse DNS sweeping. Additionally, Amass uses the IP addresses obtained during resolution to discover associated netblocks and ASNs. All the information is then used to build maps of the target networks."

* **Notebook**: Install and invoke Amass (including Go languague runtime) & Graphistry from a Google Colab notebook

## Install

In [39]:
#!pip install graphistry -q

!apt -q install golang-go libzmq3-dev
%env GOPATH=/root/go

!go get -u github.com/OWASP/Amass/...

Reading package lists...
Building dependency tree...
Reading state information...
golang-go is already the newest version (2:1.10~4ubuntu1).
libzmq3-dev is already the newest version (4.2.5-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
env: GOPATH=/root/go


## Configure

In [0]:
import graphistry
#graphistry.register(key='MY_API_KEY')
graphistry.__version__

## Crawl

* Crawl domain and store results in `data.json`
* Convert `data.json` into formatted Graphistry graph `g-viz.json`
* See [Amass](https://github.com/OWASP/Amass/blob/master/doc/user_guide.md) for further options

In [12]:
!/root/go/bin/amass -do data.json -d graphistry.com


gossip.graphistry.com 
graphistry.com 
labs.graphistry.com 
kp.graphistry.com 
riverbed.graphistry.com 
fiddlesticks.graphistry.com 
demo.graphistry.com 
spk.graphistry.com 
esto.graphistry.com 
shipyard.graphistry.com 
staging.graphistry.com 
training.graphistry.com 
estf.graphistry.com 
alpha.graphistry.com 
www.graphistry.com 
proxy-staging.graphistry.com 
deploy.graphistry.com 
dev.graphistry.com 
proxy-labs.graphistry.com 
es.graphistry.com 
qa.graphistry.com 
Average DNS queries performed: 278/sec, DNS names remaining: 16
Average DNS queries performed: 21/sec, DNS names remaining: 13
Average DNS queries performed: 14/sec, DNS names remaining: 6

OWASP Amass v2.9.9                                https://github.com/OWASP/Amass
--------------------------------------------------------------------------------
21 names discovered - alt: 2, cert: 14, dns: 1, scrape: 2, api: 2
--------------------------------------------------------------------------------
ASN: 15169 - GOOGLE - Google LL

In [0]:
!/root/go/bin/amass.viz -graphistry g-viz.json -i data.json

## Visualize!

In [40]:
import json
import pandas as pd

with open('g-viz.json') as json_file:  
    data = json.load(json_file)
    
nodes_df = pd.DataFrame(data['labels'])
edges_df = pd.DataFrame(data['graph']) 
print('# nodes', len(nodes_df))
print('# edges', len(edges_df))
nodes_df.sample(3)

# nodes 138
# edges 200


,node,pointColor,pointLabel,pointTitle,source,type
117,117,7,74.125.129.27,address: 74.125.129.27,,address
34,34,3,staging.graphistry.com,subdomain: staging.graphistry.com,Crtsh,subdomain
99,99,4,74.125.141.0/24,netblock: 74.125.141.0/24,,netblock


In [38]:
g = graphistry\
  .bind(source='src', destination='dst', edge_title='edgeTitle')\
  .bind(node='node', point_color='pointColor', point_title='pointLabel')

g.plot(edges_df, nodes_df)